1. clone git@github.com:NapthaAI/naptha-sdk.git
2. poetry install
3. make sure its available in the python env you are using

In [20]:
from naptha_sdk.user import generate_keypair, sign_consumer_id, get_public_key
from naptha_sdk.client.node import UserClient
from naptha_sdk.schemas import OrchestratorDeployment, AgentDeployment, EnvironmentDeployment, KBDeployment, NodeConfigUser, OrchestratorRunInput

In [9]:
public_key, private_key = generate_keypair()
public_key, private_key

('7f45e4ecbb55c5fc65afe2ea6f5049f47855b891216c2398f30184de781e63b702825337655c7edfb0e3db172bb6858082113bd855c363648f57a0c0e86729bb',
 '/Users/arshath/play/naptha-sdk/private_key.pem')

In [11]:
base_url = "http://node1.naptha.ai:7001"
client = UserClient(node=NodeConfigUser(ip='node1.naptha.ai', http_port=7001))

2025-01-15 11:51:38,362 - naptha_sdk.client.node - INFO - Node URL: http://node1.naptha.ai:7001


In [14]:
# response = requests.post(f"{base_url}/user/register", json={"public_key": public_key})
# print(response.json())

user_response = await client.register_user(user_input={'public_key': public_key})

{'public_key': '7f45e4ecbb55c5fc65afe2ea6f5049f47855b891216c2398f30184de781e63b702825337655c7edfb0e3db172bb6858082113bd855c363648f57a0c0e86729bb',
 'id': 'user:7f45e4ecbb55c5fc65afe2ea6f5049f47855b891216c2398f30184de781e63b702825337655c7edfb0e3db172bb6858082113bd855c363648f57a0c0e86729bb'}

In [16]:
user_id = user_response["id"]
signature = sign_consumer_id(user_id, private_key=private_key)

In [23]:
agent_deployments = [
    AgentDeployment(
        node=NodeConfigUser(ip='node1.naptha.ai'),
    ),
    AgentDeployment(
        node=NodeConfigUser(ip='node1.naptha.ai'),
    )
]

kb_deployments = [
    KBDeployment(
        node=NodeConfigUser(ip='node1.naptha.ai'),
    )
]

orchestrator_deployment = OrchestratorDeployment(
    node=NodeConfigUser(ip='node1.naptha.ai'),
    agent_deployments=agent_deployments,
    kb_deployments=kb_deployments,
    name="multiagent_chat",
    module={"id": "orchestrator:multiagent_chat", "name": "multiagent_chat", "module_type": "orchestrator"}
)

orchestrator_run_input = OrchestratorRunInput(
    consumer_id=user_id,
    inputs={"prompt": "What is the purpose of life?"},
    deployment=orchestrator_deployment,
    signature=signature
)

orchestrator_run = await client.run_orchestrator_and_poll(orchestrator_run_input)

Run input: consumer_id='user:7f45e4ecbb55c5fc65afe2ea6f5049f47855b891216c2398f30184de781e63b702825337655c7edfb0e3db172bb6858082113bd855c363648f57a0c0e86729bb' inputs={'prompt': 'What is the purpose of life?'} deployment=OrchestratorDeployment(node=NodeConfigUser(ip='node1.naptha.ai', http_port=None, server_type=None), name='multiagent_chat', module={'id': 'orchestrator:multiagent_chat', 'name': 'multiagent_chat', 'module_type': 'orchestrator'}, config=None, agent_deployments=[AgentDeployment(node=NodeConfigUser(ip='node1.naptha.ai', http_port=None, server_type=None), name=None, module=None, config=None, data_generation_config=None, tool_deployments=None, environment_deployments=None, kb_deployments=None), AgentDeployment(node=NodeConfigUser(ip='node1.naptha.ai', http_port=None, server_type=None), name=None, module=None, config=None, data_generation_config=None, tool_deployments=None, environment_deployments=None, kb_deployments=None)], environment_deployments=None, kb_deployments=[KBDe

In [27]:
import json
json.loads(orchestrator_run.results[0])

[{'role': 'user', 'content': 'What is the purpose of life?'},
 {'role': 'assistant',
  'content': "The purpose of life is a deeply philosophical question that has been explored by various cultures, religions, and thinkers throughout history. Different perspectives offer varied interpretations:\n\n1. **Existential Perspective**: Some existentialists argue that life has no inherent purpose, and it is up to each individual to create their own meaning through experiences, choices, and personal values.\n\n2. **Religious Perspective**: Many religious traditions propose that the purpose of life is to fulfill a divine plan, seek enlightenment, serve others, or achieve spiritual growth.\n\n3. **Biological Perspective**: From a scientific viewpoint, the purpose of life can be seen as survival and reproduction, ensuring the continuation of species.\n\n4. **Philosophical Perspective**: Philosophers like Aristotle believed in the concept of eudaimonia, or flourishing, suggesting that the purpose of